# NNW-Übung 4

## 1 Mehrschichtes Netz - klassisch

### a) 
Vollziehen Sie den Code des einfachen 2-schichtigen Netzes in der Datei `NNW_P4_A1_unvollstaendig.py` nach.

In [1]:
# based on: https://github.com/eakbas/tf-svm/blob/master/plot_boundary_on_data.py
import numpy as np
import copy

from matplotlib import pyplot as plt
from matplotlib import colors


def plotTwoFeatures(X,T,pred_func):
    if X.ndim!=2:
        raise ValueError('X be a matrix (2 dimensional array).')
    tp=False
    if X.shape[0]!=2 and X.shape[1]==2: 
        X=X.T
        tp=True
    if X.shape[0]!=2:
        raise ValueError('X must contain exactly 2 features.')
        
    # determine canvas borders
    mins = np.amin(X,1); 
    mins = mins - 0.1*np.abs(mins);
    maxs = np.amax(X,1); 
    maxs = maxs + 0.1*maxs;

    ## generate dense grid
    xs,ys = np.meshgrid(np.linspace(mins[0],maxs[0],300), 
            np.linspace(mins[1], maxs[1], 300));


    # evaluate model on the dense grid
    try:
        Z = pred_func(np.c_[xs.flatten(), ys.flatten()].T);
    except:
        Z = pred_func(np.c_[xs.flatten(), ys.flatten()]);

    if tp:
        Z=Z.T
    
    if Z.ndim>1 and Z.shape[0]>1: # onehot? -> convert
        Z=Z.argmax(0)
    Z = Z.reshape(xs.shape)
    
    if T.ndim>1 and T.shape[0]>1: # onehot? -> convert
        T=T.argmax(0)
        
    # Plot the contour and training examples
    plt.contourf(xs, ys, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[0,:], X[1,:], c=T, s=50,
            cmap=colors.ListedColormap(['orange', 'blue', 'green']))
    plt.show()


ImportError: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.

Original error was: cannot import name 'multiarray'


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 22 19:43:14 2016

@author: Ivo
"""
import numpy as np
import matplotlib.pyplot as plt

#%% ErrorRate
def ErrorRate(Y,T):
    if Y.ndim==1 or Y.shape[0]==1:
        errors=Y!=T
        return errors.sum()/Y.size
    else: # für mehrere Ausgaben in one-hot Kodierung:
        errors=Y.argmax(0)!=T.argmax(0)
        return errors.sum()/Y.shape[1]
        
#%%
class MLN:
    def __init__(self,dIn,hidden,cOut): # Konstruktor
        np.random.seed(42)
        self._b1=np.zeros((hidden,1))
        self._W1=np.random.randn(hidden,dIn)/np.sqrt(dIn)
        self._b2=np.zeros((cOut,1))
        self._W2=np.random.randn(cOut,hidden)/np.sqrt(dIn)
        if cOut==1:
            self.neuron=self.ythreshold
        else:
            self.neuron=self.ythresholdMult
    def net1(self,X):
         return self._W1.dot(X)+self._b1
    def z(self,X):
         return np.tanh(self.net1(X))
    def net2(self,Z):
         return self._W2.dot(Z)+self._b2
    def y(self,X):
        return self.net2(self.z(X))

    def ythreshold(self,X):
        return self.y(X)>=0
    
    def onehot(self,T):
        e=np.identity(self._W2.shape[0])
        return e[:,T.astype(int)]
    def ythresholdMult(self,X):
        return self.onehot(self.y(X).argmax(0))

    def backprop(self, X, T, Y, Z):
        N = X.shape[1] 
        deltak = Y-T
        deltaW2 = deltak.dot(Z.T)
        deltaB2 = deltak.dot(np.ones([1,N]).T); # entspricht np.sum(deltak,1)
  
        deltaj = (1-Z**2)*(self._W2.T.dot(deltak));
        # z: m*N, W2: c*m, deltak:c*N; das deltaj für jedes n soll sein 
        # zugehöriges (1-z_j^2) abbekommen
        deltaW1 = deltaj.dot(X.T);
        deltaB1 = deltaj.dot(np.ones([1,N]).T); # entspricht np.sum(deltaj,1)
        return deltaW1, deltaB1, deltaW2, deltaB2
    
    def DeltaTrain(self, X, T, eta, maxIter, maxErrorRate):
        best = self;
        bestError = 2;
        bestIt = 0;
        N=X.shape[1]    # Anzahl Trainingsdaten
        x0 = np.ones(N)[np.newaxis]
        plt.ion() # interactive mode on
        for it in range(maxIter):
            Z = self.z(X)
            Y = self.neuron(X)
            err = ErrorRate(Y, T)
            if (it%20) == 0:
                print('#{} err:{}\n{}\n{}\n{}\n{}'.format(it,err,self._W1,self._b1,self._W2,self._b2))
                plotTwoFeatures(X,T,self.neuron)
                #älteres python: plt.pause(0.05) # warte auf GUI event loop
            if err<bestError:
                bestError = err
                best = copy.copy(self)
                bestIt = it
            if err <= maxErrorRate:
                break
            deltaW1, deltaB1, deltaW2, deltaB2 = self.backprop(X,T,Y,Z)
            self._W1-=eta*deltaW1/N
            self._b1-=eta*deltaB1/N
            self._W2-=eta*deltaW2/N
            self._b2-=eta*deltaB2/N
        self._W1=best._W1
        self._b1=best._b1
        self._W2=best._W2
        self._b2=best._b2
        print('#{} err:{}\n{}\n{}\n{}\n{}'.format(it,err,self._W1,self._b1,self._W2,self._b2))
        plotTwoFeatures(X,T,self.neuron)
        #älteres python: plt.pause(0.05) # warte auf GUI event loop
        return bestError, bestIt

#%% Iris-Daten Laden
iris = np.loadtxt("iris.csv",delimiter=',')
X=iris[:,0:4].T
T=iris[:,4]

#%% Training mit 2 Iris-Blütenarten, Merkmale 0 und 1
plt.figure()
slnIris = MLN(2,1,1) # HU: 1 und 5 gut, 2,3,8 schlecht
slnIris.DeltaTrain(X[:2,0:100],T[0:100],0.01,2000,0.00)
slnIris.neuron(X[:2,:])
#%% Training mit 3 Iris-Blütenarten, Merkmale 2 und 3
plt.figure()
slnIris = MLN(2,5,3)
slnIris.DeltaTrain(X[2:4,:],slnIris.onehot(T),0.01,2000,0.04)
slnIris.neuron(X[2:4,:]).argmax(0)



# 2 Einstieg in Keras: Klasse Sequential


## a)
Importieren Sie Keras mit `from tensorflow import keras`.

In [ ]:
from tensorflow import keras